In [263]:
import numpy as np
import pandas as pd
import glob

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
import string

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [54]:
def create_dataframe(folder_name):
    
    dataset = pd.DataFrame(columns=['caption', 'label'])

    folders = glob.glob('./dataset/{}/sentences/*'.format(folder_name))
    for folder in folders:
        classes = glob.glob('{}/*.txt'.format(folder))
        for caption in classes:
            with open(caption, 'r') as file:
                inside_file = file.read()
                label = str(folder).split('\\')[1]
                for line in inside_file.split('\n'):
                    if line:
                        dataset = dataset.append({'caption': line, 'label': label}, ignore_index=True)
    return dataset

In [55]:
train = create_dataframe('train')
train.head()

,caption,label
0,Two gentleman talking in front of propeller pl...,aeroplane
1,Two men are conversing next to a small airplane.,aeroplane
2,Two men talking in front of a plane,aeroplane
3,Two men talking in front of a small plane.,aeroplane
4,Two men talk while standing next to a small pa...,aeroplane


In [56]:
test = create_dataframe('test')
test.head()

,caption,label
0,An airplane facing the camera.,aeroplane
1,A plane is sitting on the cement at a small ai...,aeroplane
2,Front of a twin engine propeller airplane.,aeroplane
3,Front view of a propeller airplane parked on a...,aeroplane
4,The historic planes sits in its place.,aeroplane


In [142]:
# import spacy

# #I'm gonna use medium size of this model
# nlp = spacy.load('en_core_web_md')

In [143]:
# def spacy_semantic_similarity(sentence1, sentence2):
#     sentence1 = nlp(sentence1)
#     sentence2 = nlp(sentence2)
    
#     return sentence1.similarity(sentence2)

In [295]:
def nlprocess(dataset):
    
    preprocessed_comments = []
    for _, (caption, label) in dataset.iterrows():

        #Lowercase
        caption = caption.lower()

        # Removing Punctuation
        caption = "".join([char for char in caption if char not in string.punctuation])

        # Word Tokenization
        caption = word_tokenize(caption)

        # Stopword Filtering
        stop_words = stopwords.words('english')
        caption = [word for word in caption if word not in stop_words]

        # Filter 
    #     caption = pos_tag(caption)
    #     print(caption)
    #     caption = [word for word, tag in caption if tag not in ('JJ', 'CD', 'VB', 'VBP', 'VBG', 'VBD', 'VBN')]


        # Stemming
    #     porter = PorterStemmer()
    #     caption = [porter.stem(word) for word in caption]


        caption = ' '.join(caption)

        preprocessed_comments.append(caption)

    return preprocessed_comments

In [296]:
preprocessed_captions_train = nlprocess(train)
preprocessed_captions_test = nlprocess(test)

In [287]:
X_train = preprocessed_captions_train
X_test = preprocessed_captions_test

y_train = train.label
y_test = test.label

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', KNeighborsClassifier(n_neighbors=50))])



text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

   aeroplane       0.84      0.88      0.86       100
     bicycle       0.63      0.76      0.69       100
        bird       0.91      0.74      0.82       100
        boat       0.82      0.84      0.83       100
         bus       0.64      0.89      0.74       100
         car       0.61      0.42      0.50       100
         cat       0.67      0.87      0.76       100
       chair       0.37      0.18      0.24       101
         cow       0.80      0.91      0.85       100
 diningtable       0.60      0.84      0.70       100
         dog       0.54      0.66      0.59       100
       horse       0.70      0.88      0.78       100
   motorbike       0.77      0.72      0.74       100
      person       0.23      0.10      0.14       100
 pottedplant       0.64      0.32      0.43       100
       sheep       0.89      0.82      0.85       100
        sofa       0.56      0.58      0.57       100
       train       0.82    